In [ ]:
from pybgpkit_parser import Parser

year="2024"
month=5

bgp_table = []

parser = Parser(url=f"data/rib.{year}{month:02d}01.0000")

for item in parser:
    if item['origin'] == 'EGP':
        as_path_list = item['as_path'].split()
        for idx, val in enumerate(as_path_list):
            if val[0] == '{' and val[-1] == '}':
                as_path_list[idx] = int(val[1:-1])
            else:
                as_path_list[idx] = int(val)
        bgp_table.append({'prefix':item['prefix'], 'as_path':as_path_list})

print("BGP Table row:", len(bgp_table))

In [ ]:
import redis

rd = redis.StrictRedis(host='localhost', port=6379, decode_responses=True)

In [ ]:
rd.flushall()

def set_as_path(as_path, idx):
    parent_asn = as_path[idx]
    asn = as_path[idx - 1]
    rd.set(f"path_parent:{asn}", parent_asn)
    rd.sadd(f"path_child:{parent_asn}", asn)
    if idx > 1:
        set_as_path(as_path, idx - 1)

def set_dest_as_path(prefix, as_path):
    rd.set(f"prefix:{prefix}", int(as_path[-1]))
    length = len(as_path)
    if length > 1:
        set_as_path(as_path, length - 1)

cnt = 0
for item in bgp_table:
    as_path = item['as_path']
    cnt += len(as_path)
    set_dest_as_path(item['prefix'], as_path)

print("AS Path Node:", cnt)
print("Prefix:", len(rd.keys('prefix:*')))
print("Tree Node:", len(rd.keys("path_parent:*")))

In [ ]:
updates = []
for day in range(1, 32):
    for hour in range(0, 24):
        for minute in range(0, 60, 15):
            date = f"{year}{month:02d}{day:02d}.{hour:02d}{minute:02d}"
            print(date)
            parser = Parser(url=f"data/updates.{date}")
            for item in parser:
                if item['origin'] == 'EGP':
                    as_path_list = item['as_path'].split()
                    for idx, val in enumerate(as_path_list):
                        if val[0] == '{' and val[-1] == '}':
                            as_path_list[idx] = int(val[1:-1])
                        else:
                            as_path_list[idx] = int(val)
                    updates.append({'prefix':item['prefix'], 'as_path':as_path_list})
len("Update Message:", updates)

In [ ]:
def update_as_path(as_path, idx):
    parent_asn = as_path[idx]
    asn = as_path[idx - 1]
    rd.set(f"path_parent:{asn}", parent_asn)
    rd.sadd(f"path_child:{parent_asn}", asn)
    if idx > 1:
        update_as_path(as_path, idx - 1)

def update_dest_as_path(prefix, as_path):
    rd.set(f"prefix:{prefix}", int(as_path[-1]))
    length = len(as_path)
    if length > 1:
        update_as_path(as_path, length - 1)

new_prefix = 0
update_dbgpsec_row = 0
validate_bgpsec = 0
validate_dbgpsec = 0

for item in updates:
    as_path = item['as_path']
    origin = rd.get(f"prefix:{item['prefix']}")
    length = len(as_path)
    validate_bgpsec += length
    if origin is None:
        new_prefix += 1
        validate_dbgpsec += length
        update_dest_as_path(item['prefix'], as_path)
    elif int(origin) != as_path[-1]:
        continue
    idx = len(as_path) - 1
    while idx > 1:
        is_same = rd.sismember(f"path_child:{as_path[idx]}", as_path[idx - 1])
        if is_same:
            idx -= 1
        else:
            update_dbgpsec_row += 1
            validate_dbgpsec += len(as_path) - idx
            update_as_path(as_path, idx)
            break

print("New Prefix:", new_prefix)
print("Update Row When use DBGPsec:", update_dbgpsec_row)
print("Validate Sign When use BGPsec:", validate_bgpsec)
print("Validate Sign When use DBGPsec:", validate_dbgpsec)